In [ ]:
import numpy as np
import pickle
from glob import glob 
import os
import sys
sys.path.append('/home/kvulic/Vulic/cmos_toolbox_w_spike_sorter/')
import re
from src.cmos_plotter.Firing_metric_plotter import *

In [ ]:
MAIN_PATH = '/itet-stor/kvulic/neuronies/single_neurons/1_Subprojects/Neurons_As_DNNs/3_Processed_Data/March2025_heart/biTE_stimulation/Full_dataset/Full_firing_files'

In [ ]:
filename = 'ID2184_N5_DIV7_DATE20250321_1210_spontaneous_reseed.raw_processed.pkl'
data = pickle.load(open(os.path.join(MAIN_PATH, f'Sorters/{filename}'), 'rb'))
print(data.keys())

In [ ]:
# Find all pickle files
pickle_files = glob(os.path.join(MAIN_PATH,"*.pkl"))
print(pickle_files)

# Initialize lists for both types of results
all_extremum_results = []
all_spikemat_results = []

# Process each file
for file in pickle_files:
    print(f"Processing {file}...")
    extremum_results, spikemat_results = process_spike_data(file)
    all_extremum_results.extend(extremum_results)
    all_spikemat_results.extend(spikemat_results)


In [ ]:
#save to pickle
with open(os.path.join(MAIN_PATH,'all_extremum_results.pkl'), 'wb') as f:
    pickle.dump(all_extremum_results, f)

In [ ]:
MAIN_PATH = '/itet-stor/kvulic/neuronies/single_neurons/1_Subprojects/Neurons_As_DNNs/3_Processed_Data/March2025_heart/'
save_path = os.path.join(MAIN_PATH, 'Results')
with open(os.path.join(save_path, 'combined_unit_metrics.pkl'), 'rb') as f:
    df_extremum = pd.DataFrame(pickle.load(f))
df_extremum['area'] =  df_extremum['area'].astype(float)
df_1 = df_extremum


In [ ]:
MAIN_PATH = '/itet-stor/kvulic/neuronies/single_neurons/1_Subprojects/Neurons_As_DNNs/3_Processed_Data/March2025_heart/'
save_path = os.path.join(MAIN_PATH, 'biTE_stimulation/Full_dataset/Full_firing_files')
with open(os.path.join(save_path, 'all_extremum_results.pkl'), 'rb') as f:
    df_extremum = pd.DataFrame(pickle.load(f))
df_extremum['area'] =  df_extremum['area'].astype(float)

In [ ]:
def custom_lineplot(data, x_metric, y_metric, axes, label, fontsize=16, labelsize=14, xlim = None, ylim = None, save_path=None):
    plt.rcParams['xtick.bottom'] = True    
    plt.rcParams['ytick.left'] = True    
    plt.rcParams['font.family'] = 'Arial'    
    sns.lineplot(data=data, x=x_metric, y=y_metric, color="dimgrey", markers=True, estimator="mean", 
                 err_style="bars", legend=True, zorder=1, ax=axes, dashes=False, marker="o", ms=10, linewidth=0.5, errorbar=("ci", 95))        
    axes.set_ylabel(label, fontsize=fontsize)    
    axes.set_xlabel("Day In Vitro", fontsize=fontsize)    
    axes.grid(True, linestyle=':', alpha=0.7)    
    axes.tick_params(axis='both', which='major', labelsize=labelsize, width=1.5, length=8)
    if xlim is not None:
        axes.set_xlim(xlim)
    if ylim is not None:
        axes.set_ylim(ylim)
    if save_path:
        plt.savefig(os.path.join(save_path, f'{label[:-3]}.png'), dpi=300)
        plt.savefig(os.path.join(save_path, f'{label[:-3]}.pdf'), dpi=300)
    

In [ ]:
save_path = os.path.join(MAIN_PATH, 'Results')
df_extremum = df_extremum[df_extremum['div'].isin([23, 65, 107, 129, 147])]

In [ ]:
def multi_dataframe_lineplot(data_list, x_metrics, y_metrics, axes, labels, colors, fontsize=16, labelsize=14, xlim=None, ylim=None, save_path=None, log_scale=False):
    plt.rcParams['xtick.bottom'] = True    
    plt.rcParams['ytick.left'] = True    
    plt.rcParams['font.family'] = 'Arial'
    
    # Plot each dataframe with its own color
    for data, x_metric, y_metric, label, color in zip(data_list, x_metrics, y_metrics, labels, colors):
        sns.lineplot(data=data, x=x_metric, y=y_metric, color=color, markers=True, estimator="mean", 
                    err_style="bars", label=label, zorder=1, ax=axes, dashes=False, 
                    marker="o", ms=10, linewidth=0.5, errorbar=("ci", 95))
    
    # Set common axes properties
    axes.set_ylabel(labels[0], fontsize=fontsize)  # Using the first label for y-axis title
    axes.set_xlabel("Log (Day In Vitro)", fontsize=fontsize)    
    axes.grid(True, linestyle=':', alpha=0.7)    
    axes.tick_params(axis='both', which='major', labelsize=labelsize, width=1.5, length=8)
    axes.legend()
    
    if log_scale:
        axes.set_xscale('log')  # Set x-axis to log scale
    

    if xlim is not None:
        axes.set_xlim(xlim)
    if ylim is not None:
        axes.set_ylim(ylim)
    
    if save_path:
        plt.savefig(os.path.join(save_path, f'firing_rate_combined.png'), dpi=300)
        plt.savefig(os.path.join(save_path, f'firing_rate_combined.pdf'), dpi=300)

In [ ]:
# Filter data if needed (like in your example with speeds)
df_1 = pd.DataFrame(df_1)
df_2 = pd.DataFrame(df_2)
df_2 = df_2[df_2['div'] > 10]
df_1 = df_1[df_1['div'].isin([23, 65, 107, 129, 147])]

# Create figure and axes
fig, ax = plt.subplots(figsize=(10, 6))

# Define colors - dimgray for first and a lighter gray for second
colors = ['dimgray', '#A9A9A9']  # dimgray and darkgray

# Call the modified function with both dataframes
multi_dataframe_lineplot(
    data_list=[df_1, df_2],
    x_metrics=[df_1['div'], df_2['div']],
    y_metrics=[df_1['firing_rate'], df_2['firing_rate']],
    axes=ax,
    labels=['Neuron Firing Rate [Hz] - Group 1', 'Neuron Firing Rate [Hz] - Group 2'],
    colors=colors,
    fontsize=16, 
    labelsize=14, 
    xlim=None, 
    #ylim=(0.35, 0.8), 
    save_path=save_path,
    log_scale=True
)

plt.tight_layout()
plt.show()